<a href="https://colab.research.google.com/github/Kentea-Watanabe/make_girlfriends/blob/main/GoogleAPIScraiping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 24時間で100回までしかAPIを呼び出せません。
# 17時にリセットされる

import os
from datetime import datetime
import json
from googleapiclient.discovery import build


start_index = 1

# TODO 
GOOGLE_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxx"
CUSTOM_SEARCH_ENGINE_ID = "xxxxxxxxxxxxxxxxxxxxx"

# TODO 検索ワードを変換する
KEYWORD = "かわいい 女性 海外"
KEYWORD_DIR = 'cute_girls'

# -pで存在していれば作成しないというものに変更できる
!mkdir -p {KEYWORD_DIR}

if __name__ == '__main__':
  # Google Customサーチ結果を取得
  for _ in range(10):
    s = build("customsearch","v1",
    developerKey = GOOGLE_API_KEY)
    r = s.cse().list(q = KEYWORD,
    cx = CUSTOM_SEARCH_ENGINE_ID,
    # lr = 'lang_ja',
    searchType = "image", #検索タイプ
    num = 10,
    start = start_index).execute()

    # レスポンスをjson形式で保存
    s = json.dumps(r, ensure_ascii = False, indent = 4)
    now = datetime.today().strftime("%Y%m%d%H%M%S")
    
    with open('./' + KEYWORD_DIR + "/" + KEYWORD_DIR + "_" + str(_) + '.json', mode='w') as f:
      f.write(s)
    
    start_index += 10

In [ ]:
%cd {KEYWORD_DIR}

In [ ]:
from http.client import NOT_EXTENDED
from logging import getLogger, StreamHandler, Formatter, DEBUG
import requests
import shutil
import json

LOGGER = getLogger('custom_search_api')
HANDLER = StreamHandler()
HANDLER.setLevel(DEBUG)
HANDLER.setFormatter(Formatter('%(message)s'))
LOGGER.setLevel(DEBUG)
LOGGER.addHandler(HANDLER)


# TODO imgフォルダを作成する
!mkdir -p img


# 保存する画像名を更新していくための変数
next_var = 0


for i in range(10):
  res_json = open('./' + KEYWORD_DIR + "_" + str(i) + '.json', 'r')
  res_json = json.load(res_json)

  for idx, items in enumerate(res_json["items"]):
      idx = idx + next_var
      path = "./img/" + KEYWORD_DIR + "_" + str(idx) + ".jpg"
      download_link = items['link']
      LOGGER.info(f'url:{download_link}')
      r = requests.get(download_link, stream=True)
      if r.status_code == 200:
        try:
          with open(path, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
            print("保存成功!!!")
        except:
          print("画像ではありません")
      next_var += 10